# TP4 – Projet A/B Testing avec RetailRocket
## Master AIA02-1 — Séance 4 : Analyse statistique & Décision produit

**Objectif** : Analyser un dataset e-commerce et réaliser un A/B test complet

## PARTIE 1 — Initialisation du projet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)

In [ ]:
# Chargement des données
df = pd.read_csv('events.csv')
print("Dataset chargé avec succès!")
print(f"Dimensions: {df.shape}")
print("\nAperçu des données:")
df.head(10)

In [ ]:
# Informations générales
print("Informations sur le dataset:")
print(df.info())
print("\nStatistiques descriptives:")
print(df.describe())

In [ ]:
# Types d'événements
print("Répartition des événements:")
print(df['event'].value_counts())
print("\nProportions:")
print(df['event'].value_counts(normalize=True) * 100)

## PARTIE 2 — Nettoyage & Exploration

In [ ]:
# Vérification des valeurs manquantes
print("Valeurs manquantes par colonne:")
print(df.isnull().sum())
print("\nPourcentage de valeurs manquantes:")
print((df.isnull().sum() / len(df)) * 100)

In [ ]:
# Filtrer uniquement view et addtocart
df_filtered = df[df['event'].isin(['view', 'addtocart'])].copy()
print(f"Dataset original: {len(df)} lignes")
print(f"Dataset filtré: {len(df_filtered)} lignes")
print(f"Réduction: {((len(df) - len(df_filtered)) / len(df) * 100):.2f}%")

In [ ]:
# Statistiques après filtrage
print("\nNombre d'événements par type:")
print(df_filtered['event'].value_counts())
print("\nNombre de visiteurs uniques:")
print(f"{df_filtered['visitorid'].nunique():,}")
print("\nNombre de produits uniques:")
print(f"{df_filtered['itemid'].nunique():,}")

In [ ]:
# Échantillonnage si nécessaire (optionnel)
# Si le dataset est trop volumineux, on peut échantillonner
if len(df_filtered) > 500000:
    print("Dataset volumineux détecté. Échantillonnage en cours...")
    sample_visitors = df_filtered['visitorid'].unique()
    np.random.seed(42)
    sample_visitors = np.random.choice(sample_visitors, size=int(len(sample_visitors)*0.3), replace=False)
    df_filtered = df_filtered[df_filtered['visitorid'].isin(sample_visitors)]
    print(f"Nouveau dataset: {len(df_filtered)} lignes")
else:
    print("Pas d'échantillonnage nécessaire.")

## PARTIE 3 — Simulation d'un A/B Test

In [ ]:
# Extraction des visiteurs uniques
unique_visitors = df_filtered['visitorid'].unique()
print(f"Nombre de visiteurs uniques: {len(unique_visitors):,}")

In [ ]:
# Randomisation des visiteurs en groupes A et B (50/50)
np.random.seed(42)  # Pour reproductibilité
visitor_groups = pd.DataFrame({
    'visitorid': unique_visitors,
    'group': np.random.choice(['A', 'B'], size=len(unique_visitors), p=[0.5, 0.5])
})

print("Répartition des groupes:")
print(visitor_groups['group'].value_counts())
print("\nProportions:")
print(visitor_groups['group'].value_counts(normalize=True) * 100)

In [ ]:
# Ajout de la colonne group au dataset principal
df_ab = df_filtered.merge(visitor_groups, on='visitorid', how='left')
print("Colonne 'group' ajoutée avec succès!")
print(f"\nVérification: {df_ab['group'].isnull().sum()} valeurs manquantes")
df_ab.head()

In [ ]:
# Vérification de la répartition des événements par groupe
print("Répartition des événements par groupe:")
print(pd.crosstab(df_ab['event'], df_ab['group']))
print("\nProportions normalisées:")
print(pd.crosstab(df_ab['event'], df_ab['group'], normalize='columns') * 100)

## PARTIE 4 — KPI : Add-to-Cart Rate

In [ ]:
# Calcul des métriques pour le groupe A
df_group_a = df_ab[df_ab['group'] == 'A']
views_a = len(df_group_a[df_group_a['event'] == 'view'])
addtocart_a = len(df_group_a[df_group_a['event'] == 'addtocart'])
rate_a = (addtocart_a / views_a) * 100

print("GROUPE A:")
print(f"  Views: {views_a:,}")
print(f"  Add-to-cart: {addtocart_a:,}")
print(f"  Taux d'ajout au panier: {rate_a:.4f}%")

In [ ]:
# Calcul des métriques pour le groupe B
df_group_b = df_ab[df_ab['group'] == 'B']
views_b = len(df_group_b[df_group_b['event'] == 'view'])
addtocart_b = len(df_group_b[df_group_b['event'] == 'addtocart'])
rate_b = (addtocart_b / views_b) * 100

print("GROUPE B:")
print(f"  Views: {views_b:,}")
print(f"  Add-to-cart: {addtocart_b:,}")
print(f"  Taux d'ajout au panier: {rate_b:.4f}%")

In [ ]:
# Comparaison
diff_absolute = rate_b - rate_a
diff_relative = ((rate_b - rate_a) / rate_a) * 100

print("\n" + "="*50)
print("COMPARAISON DES GROUPES")
print("="*50)
print(f"Groupe A: {rate_a:.4f}%")
print(f"Groupe B: {rate_b:.4f}%")
print(f"Différence absolue: {diff_absolute:.4f} points de pourcentage")
print(f"Différence relative: {diff_relative:.2f}%")

if diff_absolute > 0:
    print(f"\nLe groupe B est supérieur de {diff_absolute:.4f} points.")
elif diff_absolute < 0:
    print(f"\nLe groupe A est supérieur de {abs(diff_absolute):.4f} points.")
else:
    print("\nLes deux groupes ont des taux identiques.")

In [ ]:
# Visualisation
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1: Taux de conversion
groups = ['Groupe A', 'Groupe B']
rates = [rate_a, rate_b]
colors = ['#3498db', '#e74c3c']

ax[0].bar(groups, rates, color=colors, alpha=0.7, edgecolor='black')
ax[0].set_ylabel('Taux d\'ajout au panier (%)', fontsize=12)
ax[0].set_title('Comparaison des taux de conversion', fontsize=14, fontweight='bold')
ax[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(rates):
    ax[0].text(i, v + 0.01, f'{v:.4f}%', ha='center', va='bottom', fontweight='bold')

# Graphique 2: Volume d'événements
events_data = pd.DataFrame({
    'Groupe': ['A', 'A', 'B', 'B'],
    'Type': ['Views', 'Add-to-cart', 'Views', 'Add-to-cart'],
    'Count': [views_a, addtocart_a, views_b, addtocart_b]
})

events_pivot = events_data.pivot(index='Type', columns='Groupe', values='Count')
events_pivot.plot(kind='bar', ax=ax[1], color=colors, alpha=0.7, edgecolor='black')
ax[1].set_ylabel('Nombre d\'événements', fontsize=12)
ax[1].set_title('Volume d\'événements par groupe', fontsize=14, fontweight='bold')
ax[1].legend(title='Groupe')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=0)
ax[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('ab_test_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Graphique sauvegardé: ab_test_comparison.png")

## PARTIE 5 — Test statistique : Test de proportion

In [ ]:
# Hypothèses du test
print("TEST STATISTIQUE DE PROPORTION Z-TEST")
print("="*50)
print("H0: Les taux de conversion A et B sont identiques (p_A = p_B)")
print("H1: Les taux de conversion A et B sont différents (p_A ≠ p_B)")
print("Seuil de significativité: α = 0.05")
print("="*50)

In [ ]:
# Données pour le test
n_a = views_a  # Nombre total de views groupe A
n_b = views_b  # Nombre total de views groupe B
x_a = addtocart_a  # Nombre de succès (addtocart) groupe A
x_b = addtocart_b  # Nombre de succès (addtocart) groupe B

p_a = x_a / n_a  # Proportion groupe A
p_b = x_b / n_b  # Proportion groupe B

print("Données du test:")
print(f"Groupe A: {x_a}/{n_a} = {p_a:.6f}")
print(f"Groupe B: {x_b}/{n_b} = {p_b:.6f}")

In [ ]:
# Calcul de la proportion combinée (pooled proportion)
p_pooled = (x_a + x_b) / (n_a + n_b)
print(f"\nProportion combinée (pooled): {p_pooled:.6f}")

In [ ]:
# Calcul de l'erreur standard
se = np.sqrt(p_pooled * (1 - p_pooled) * (1/n_a + 1/n_b))
print(f"Erreur standard: {se:.6f}")

In [ ]:
# Calcul du Z-score
z_score = (p_b - p_a) / se
print(f"\nZ-score: {z_score:.4f}")

In [ ]:
# Calcul de la p-value (test bilatéral)
p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))
print(f"P-value (test bilatéral): {p_value:.6f}")

In [ ]:
# Interprétation
alpha = 0.05
print("\n" + "="*50)
print("RÉSULTATS DU TEST")
print("="*50)
print(f"Z-score: {z_score:.4f}")
print(f"P-value: {p_value:.6f}")
print(f"Seuil α: {alpha}")

if p_value < alpha:
    print(f"\nCONCLUSION: p-value ({p_value:.6f}) < α ({alpha})")
    print("➜ On REJETTE l'hypothèse nulle H0")
    print("➜ La différence entre les groupes A et B est STATISTIQUEMENT SIGNIFICATIVE")
    
    if p_b > p_a:
        print(f"➜ Le groupe B est significativement MEILLEUR que A ({rate_b:.4f}% vs {rate_a:.4f}%)")
        print(f"➜ Amélioration relative: +{diff_relative:.2f}%")
    else:
        print(f"➜ Le groupe A est significativement MEILLEUR que B ({rate_a:.4f}% vs {rate_b:.4f}%)")
        print(f"➜ Amélioration relative: +{abs(diff_relative):.2f}%")
else:
    print(f"\nCONCLUSION: p-value ({p_value:.6f}) >= α ({alpha})")
    print("➜ On NE PEUT PAS REJETER l'hypothèse nulle H0")
    print("➜ La différence entre les groupes A et B N'est PAS statistiquement significative")
    print("➜ Les deux groupes peuvent être considérés comme équivalents")

In [ ]:
# Vérification avec scipy.stats
print("\n" + "="*50)
print("VÉRIFICATION AVEC scipy.stats.proportions_ztest")
print("="*50)

from statsmodels.stats.proportion import proportions_ztest

count = np.array([x_b, x_a])
nobs = np.array([n_b, n_a])
z_stat, p_val = proportions_ztest(count, nobs, alternative='two-sided')

print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_val:.6f}")
print(f"\nCohérence avec calculs manuels: {'OUI' if abs(z_stat - z_score) < 0.01 else 'NON'}")

## PARTIE 6 — Analyse Business

In [ ]:
print("ANALYSE BUSINESS")
print("="*70)

print("\n1. DIFFÉRENCE ENTRE LES TAUX")
print("-" * 70)
print(f"   Groupe A: {rate_a:.4f}%")
print(f"   Groupe B: {rate_b:.4f}%")
print(f"   Différence absolue: {abs(diff_absolute):.4f} points")
print(f"   Différence relative: {abs(diff_relative):.2f}%")

print("\n2. SIGNIFICATIVITÉ STATISTIQUE")
print("-" * 70)
if p_value < alpha:
    print(f"   ✓ Différence SIGNIFICATIVE (p={p_value:.6f} < 0.05)")
    print(f"   ✓ Confiance: {(1-p_value)*100:.2f}%")
else:
    print(f"   ✗ Différence NON significative (p={p_value:.6f} >= 0.05)")
    print(f"   ✗ Risque d'erreur trop élevé")

print("\n3. RISQUE D'ERREUR")
print("-" * 70)
print(f"   Risque de faux positif (Type I): {alpha*100}%")
print(f"   P-value observée: {p_value:.6f}")
if p_value < alpha:
    print(f"   Probabilité que la différence soit due au hasard: {p_value*100:.4f}%")
else:
    print(f"   Probabilité élevée que la différence soit due au hasard")

print("\n4. DÉCISION RECOMMANDÉE")
print("-" * 70)

if p_value < alpha:
    if p_b > p_a:
        print("   ✓ DÉPLOYER LA VARIANTE B EN PRODUCTION")
        print(f"   ✓ Impact attendu: +{diff_relative:.2f}% d'ajouts au panier")
        print(f"   ✓ Gain absolu: +{diff_absolute:.4f} points")
        
        # Estimation de l'impact business
        if views_b > 0:
            gain_per_1000_views = (diff_absolute / 100) * 1000
            print(f"   ✓ Pour 1000 vues: ~{gain_per_1000_views:.1f} ajouts au panier supplémentaires")
    else:
        print("   ✓ CONSERVER LA VARIANTE A (meilleure performance)")
        print(f"   ✗ Ne pas déployer B: baisse de {abs(diff_relative):.2f}%")
else:
    print("   ⚠ AUCUN CHANGEMENT RECOMMANDÉ")
    print("   ⚠ Différence non significative statistiquement")
    print("   ⚠ Options:")
    print("      1. Prolonger le test pour collecter plus de données")
    print("      2. Augmenter la taille d'échantillon")
    print("      3. Tester une variante C avec des changements plus importants")

print("\n5. RECOMMANDATIONS PRODUIT")
print("-" * 70)

if p_value < alpha and p_b > p_a:
    print("   • Déployer progressivement B (10% → 50% → 100% du trafic)")
    print("   • Monitorer les métriques post-déploiement pendant 2 semaines")
    print("   • Vérifier l'impact sur les transactions finales")
    print("   • Analyser les segments d'utilisateurs (nouveaux vs récurrents)")
    print("   • Documenter les learnings pour futurs A/B tests")
elif p_value < alpha and p_a > p_b:
    print("   • Ne pas déployer B, conserver A")
    print("   • Analyser pourquoi B performe moins bien")
    print("   • Itérer sur le design de B pour une nouvelle version C")
    print("   • Tester des hypothèses alternatives")
else:
    print("   • Prolonger le test de 1-2 semaines supplémentaires")
    print("   • Si toujours pas significatif: tester des changements plus impactants")
    print("   • Analyser les données qualitatives (heatmaps, sessions recordings)")
    print("   • Considérer des segments spécifiques où B pourrait être meilleur")
    print("   • Budget: prioriser d'autres optimisations à plus fort impact")

print("\n" + "="*70)

## Synthèse des résultats

In [ ]:
# Tableau récapitulatif
summary_df = pd.DataFrame({
    'Métrique': ['Nombre de views', 'Nombre d\'add-to-cart', 'Taux de conversion (%)', 
                 'Différence absolue', 'Différence relative (%)', 'Z-score', 'P-value', 'Significatif?'],
    'Groupe A': [f"{views_a:,}", f"{addtocart_a:,}", f"{rate_a:.4f}%", '-', '-', '-', '-', '-'],
    'Groupe B': [f"{views_b:,}", f"{addtocart_b:,}", f"{rate_b:.4f}%", 
                 f"{diff_absolute:.4f} pts", f"{diff_relative:.2f}%", 
                 f"{z_score:.4f}", f"{p_value:.6f}", 
                 'OUI' if p_value < alpha else 'NON']
})

print("\nTABLEAU RÉCAPITULATIF")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)

In [ ]:
# Sauvegarde des résultats
summary_df.to_csv('ab_test_results.csv', index=False)
print("\nRésultats sauvegardés dans: ab_test_results.csv")

## PARTIE 7 — Génération du rapport PDF

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

# Création du rapport PDF
pdf_filename = 'Rapport_AB_Test_RetailRocket.pdf'

with PdfPages(pdf_filename) as pdf:
    # Page 1: Résultats principaux
    fig = plt.figure(figsize=(8.5, 11))
    fig.text(0.5, 0.95, 'RAPPORT A/B TEST - RETAILROCKET', 
             ha='center', fontsize=16, fontweight='bold')
    fig.text(0.5, 0.92, f'Master AIA02-1 - {datetime.now().strftime("%d/%m/%Y")}', 
             ha='center', fontsize=10)
    
    # Section 1: Contexte
    y_pos = 0.88
    fig.text(0.1, y_pos, '1. CONTEXTE DU TEST', fontsize=12, fontweight='bold')
    y_pos -= 0.03
    context_text = f"""Objectif: Comparer deux variantes (A et B) de l'interface e-commerce
KPI mesuré: Taux d'ajout au panier (Add-to-Cart Rate)
Période: Analyse du dataset RetailRocket events.csv
Population: {len(unique_visitors):,} visiteurs uniques
Événements analysés: {len(df_filtered):,} (views et add-to-cart)"""
    fig.text(0.1, y_pos, context_text, fontsize=9, verticalalignment='top')
    
    # Section 2: Méthodologie
    y_pos -= 0.15
    fig.text(0.1, y_pos, '2. MÉTHODOLOGIE', fontsize=12, fontweight='bold')
    y_pos -= 0.03
    method_text = f"""• Randomisation: Attribution aléatoire 50/50 des visiteurs aux groupes A et B
• Test statistique: Test Z de comparaison de proportions
• Seuil de significativité: α = 0.05 (95% de confiance)
• Hypothèses:
  - H0: Taux de conversion A = Taux de conversion B
  - H1: Taux de conversion A ≠ Taux de conversion B"""
    fig.text(0.1, y_pos, method_text, fontsize=9, verticalalignment='top')
    
    # Section 3: KPI
    y_pos -= 0.18
    fig.text(0.1, y_pos, '3. RÉSULTATS - KPI', fontsize=12, fontweight='bold')
    y_pos -= 0.03
    
    kpi_text = f"""┌─────────────────────────────────────────────────────────┐
│  GROUPE A                    │  GROUPE B                 │
├─────────────────────────────────────────────────────────┤
│  Views: {views_a:,}              │  Views: {views_b:,}           │
│  Add-to-cart: {addtocart_a:,}        │  Add-to-cart: {addtocart_b:,}     │
│  Taux: {rate_a:.4f}%              │  Taux: {rate_b:.4f}%           │
└─────────────────────────────────────────────────────────┘

Différence absolue: {abs(diff_absolute):.4f} points
Différence relative: {abs(diff_relative):.2f}%
Meilleur groupe: {'B' if p_b > p_a else 'A'}"""
    fig.text(0.1, y_pos, kpi_text, fontsize=8, verticalalignment='top', family='monospace')
    
    # Section 4: Test statistique
    y_pos -= 0.20
    fig.text(0.1, y_pos, '4. TEST STATISTIQUE', fontsize=12, fontweight='bold')
    y_pos -= 0.03
    stat_text = f"""Z-score: {z_score:.4f}
P-value: {p_value:.6f}
Seuil α: {alpha}

Conclusion: {'DIFFÉRENCE SIGNIFICATIVE' if p_value < alpha else 'DIFFÉRENCE NON SIGNIFICATIVE'}
➜ {'On REJETTE H0' if p_value < alpha else 'On NE PEUT PAS REJETER H0'}
➜ Confiance: {(1-p_value)*100:.2f}%"""
    fig.text(0.1, y_pos, stat_text, fontsize=9, verticalalignment='top')
    
    # Section 5: Décision
    y_pos -= 0.18
    fig.text(0.1, y_pos, '5. DÉCISION FINALE', fontsize=12, fontweight='bold')
    y_pos -= 0.03
    
    if p_value < alpha and p_b > p_a:
        decision = f"""✓ DÉPLOYER LA VARIANTE B EN PRODUCTION
  Justification: Amélioration significative de +{diff_relative:.2f}%
  Impact: +{diff_absolute:.4f} points de taux de conversion"""
    elif p_value < alpha and p_a > p_b:
        decision = f"""✓ CONSERVER LA VARIANTE A
  Justification: A performe significativement mieux
  Impact: B causerait une baisse de {abs(diff_relative):.2f}%"""
    else:
        decision = """⚠ AUCUN CHANGEMENT RECOMMANDÉ
  Justification: Différence non significative
  Action: Prolonger le test ou tester une variante C"""
    
    fig.text(0.1, y_pos, decision, fontsize=10, verticalalignment='top')
    
    # Section 6: Recommandations
    y_pos -= 0.15
    fig.text(0.1, y_pos, '6. RECOMMANDATIONS PRODUIT', fontsize=12, fontweight='bold')
    y_pos -= 0.03
    
    if p_value < alpha and p_b > p_a:
        reco = """• Déploiement progressif: 10% → 50% → 100% du trafic
• Monitoring post-déploiement sur 2 semaines
• Analyse de l'impact sur les transactions complètes
• Segmentation: impact sur nouveaux vs anciens clients"""
    else:
        reco = """• Prolonger le test pour augmenter la puissance statistique
• Analyser les données qualitatives (heatmaps, recordings)
• Tester une variante C avec des changements plus marqués
• Prioriser d'autres optimisations à plus fort impact"""
    
    fig.text(0.1, y_pos, reco, fontsize=9, verticalalignment='top')
    
    plt.axis('off')
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    
    # Page 2: Graphiques
    fig2, axes = plt.subplots(2, 1, figsize=(8.5, 11))
    fig2.suptitle('VISUALISATIONS - A/B TEST', fontsize=14, fontweight='bold', y=0.98)
    
    # Graphique 1: Taux de conversion
    groups = ['Groupe A', 'Groupe B']
    rates = [rate_a, rate_b]
    colors = ['#3498db', '#e74c3c']
    
    axes[0].bar(groups, rates, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    axes[0].set_ylabel('Taux d\'ajout au panier (%)', fontsize=11)
    axes[0].set_title('Comparaison des taux de conversion', fontsize=12, fontweight='bold', pad=15)
    axes[0].grid(axis='y', alpha=0.3, linestyle='--')
    for i, v in enumerate(rates):
        axes[0].text(i, v + max(rates)*0.01, f'{v:.4f}%', 
                     ha='center', va='bottom', fontweight='bold', fontsize=11)
    
    # Graphique 2: Distribution normale et zone de rejet
    x = np.linspace(-4, 4, 1000)
    y = stats.norm.pdf(x, 0, 1)
    
    axes[1].plot(x, y, 'k-', linewidth=2, label='Distribution normale standard')
    axes[1].fill_between(x, y, where=(x <= -1.96) | (x >= 1.96), 
                          alpha=0.3, color='red', label='Zone de rejet (α=0.05)')
    axes[1].axvline(z_score, color='blue', linestyle='--', linewidth=2, 
                    label=f'Z-score observé = {z_score:.4f}')
    axes[1].axvline(-1.96, color='red', linestyle=':', linewidth=1)
    axes[1].axvline(1.96, color='red', linestyle=':', linewidth=1)
    axes[1].set_xlabel('Z-score', fontsize=11)
    axes[1].set_ylabel('Densité de probabilité', fontsize=11)
    axes[1].set_title('Test Z bilatéral - Visualisation', fontsize=12, fontweight='bold', pad=15)
    axes[1].legend(loc='upper right', fontsize=9)
    axes[1].grid(True, alpha=0.3, linestyle='--')
    axes[1].text(0, max(y)*0.5, f'p-value = {p_value:.6f}', 
                 ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    pdf.savefig(fig2, bbox_inches='tight')
    plt.close()

print(f"\n✓ Rapport PDF généré avec succès: {pdf_filename}")
print(f"✓ Le rapport contient 2 pages avec l'analyse complète")

## Conclusion

Ce notebook a permis de réaliser un A/B test complet sur le dataset RetailRocket:

1. ✓ Chargement et exploration des données
2. ✓ Nettoyage et préparation
3. ✓ Simulation d'une randomisation correcte (A/B)
4. ✓ Calcul du KPI Add-to-Cart Rate
5. ✓ Test statistique de comparaison de proportions
6. ✓ Interprétation et décision business
7. ✓ Génération d'un rapport PDF professionnel

**Fichiers générés:**
- `ab_test_comparison.png` : Graphiques de comparaison
- `ab_test_results.csv` : Tableau récapitulatif
- `Rapport_AB_Test_RetailRocket.pdf` : Rapport complet